<h4> Functions </h4>

In [1]:
#Scripts last tested on 16 April 2020
import tagui as t
import math
import csv
import pandas as pd

In [2]:
#Get the list from MOH HCI list 
flag = 0 

def init_process():
    global flag
    
    if flag == 0:
        t.init()
        flag = 1
        print ("Tagui Just Initialised")
    else:
        print ("Tagui Re-Initialised")
        
    t.url('http://hcidirectory.sg/hcidirectory/')
    t.wait(20)

In [3]:
#search for all clinics
def search_process():
    t.click('//input[@id="search_btn"]')
    t.wait(20)

In [4]:
def copy_items(file_name):
    total_items = int(t.read('//*[@id="results"]/div/div[2]/div[1]/text()').split(' ')[5].replace(',',''))
    print('The total number of items = %d' %total_items)
    
    item_per_page = 10
    print ('Items per page = %d' %item_per_page)
    
    current_page = int(t.read('//input[@id="targetPageNo"]/@value'))
    print ('Current page number  = %d' %current_page)
    
    page_count = int(t.read('//input[@id="totalPage"]/@value'))
    print ('Number of pages = %d' %page_count)
    
    with open(file_name+'.csv', 'w', newline='',encoding="utf-8") as file: #create csv file 
        writer = csv.writer(file,delimiter=',')
        writer.writerow(["Sno", "Name","Contact Number", "Fax","Address", "Opening Hours", "Link"]) #create header
        
        for i in range(1,page_count+1):
            for j in range(1, item_per_page + 1):
                d_temp = []
                sno = (i-1) * item_per_page + j
                
                #retrieve the details
                if t.present(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="name"])/a') and sno <=total_items:
                    name = t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="name"]/a)') #retrieve name
                    
                    if t.present(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="contact_browser"])'):
                        contact = t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="contact_browser"])')
                    else:
                        contact = "N/A"
                        
                    if t.present(f'(//*[@id="results"]//div[@class="result_container"]//span[@class="tel"]/text()[preceding-sibling::br])'):
                        fax = t.read(f'(//*[@id="results"]//div[@class="result_container"]//span[@class="tel"]/text()[preceding-sibling::br])').replace('F.\xa0\xa0','')
                    else:
                        fax ="N/A"
                        
                    if t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="add"])'):
                        address = t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="add"])').replace('\t','').replace('\n','')  #retrieve contact number 
                    else:
                        address ="N/A"
                    
                    if t.present(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="time"])'):
                        opening_hours = t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="time"])') #retrieve time 
                    else:
                        opening_hours = "N/A"
                    
                    if t.present(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="name"]/a/@href)'):
                        url = t.read(f'(//*[@id="results"]//div[@class="result_container"][{j}]//span[@class="name"]/a/@href)')
                        link = 'http://hcidirectory.sg'+ url
                    else:
                        link = "N/A"
                
                else:
                    current_page = t.read('//input[@id="targetPageNo"]/@value')
                    name,contact,fax,address,opening_hours,link ="No Clinic Found","NA","NA","NA","NA",current_page
                    
                d_temp = [sno,name,contact,fax,address,opening_hours,link]
                writer.writerow(d_temp) #write to csv file
                print (d_temp)
            
            if t.present('//a[@class="next2"]'): #go to next page
                t.click('//a[@class="next2"]')
                t.wait(10)
                
    file.close()
    global flag
    flag = 0
    t.close()

In [88]:
def copy_details(url): 
    t.url(url)
    t.wait(10)
    
    #copy services
    if t.present ('//h2[contains(text(),"Details of Services")]'):
        service_count = t.count('//h2[contains(text(),"Details of Services")]/following::table[1]//td')
        service = ""
        if service_count == 1:
            service = t.read(f'//h2[contains(text(),"Details of Services")]/following::table[1]//td') 
        else:
            for i in range(1,service_count+1):
                service = t.read(f'//h2[contains(text(),"Details of Services")]/following::table[1]//td[{i}]') + ',' + service 
    else:
        service = "NA"
    
    #copy programs
    if t.present ('//h2[contains(text(),"Programmes")]'):
        program_count = t.count('//h2[contains(text(),"Programmes")]/following::table[1]//td')
        program = ""
        if program_count == 1:
            program = t.read(f'//h2[contains(text(),"Programmes")]/following::table[1]//td')  
        else:
            for i in range(1,service_count+1):
                program = t.read(f'//h2[contains(text(),"Programmes")]/following::table[1]//td[{i}]') + ',' + program 
    else:
        program = "NA"
    
    #copy fees
    if t.present ('//h2[contains(text(),"Fees")]'):
        fees = t.count('//h2[contains(text(),"Fees")]/following::table[1]//td')
    else:
        fees = "NA"
        
    return service,program,fees

<h4> Extract the List </h4>

In [ ]:
#script last run on 13 April, there were 4401 results returned 
init_process()
search_process()
copy_items("clinic_list")

<h4>Extract All the Program </h4>

In [5]:
#to initialise the program
init_process()

Tagui Just Initialised


In [39]:
path = 'data\clinic_list.csv'
clinic_list = pd.read_csv (path)
clinic_list.head(5)

,Sno,Name,Contact Number,Fax,Address,Opening Hours,Link
0,1,DA CLINIC @ BISHAN,69541012.0,69541012,BLK 501 BISHAN STREET 11 ...,Eve of Chinese New Year (24/01/2020) : 09:00 a...,http://hcidirectory.sg/hcidirectory/clinic.do?...
1,2,PACIFIC ADVANCE RENAL CARE (CHOA CHU KANG),68930617.0,69541012,BLK 488B CHOA CHU KANG AVENUE 5 ...,Please call the clinic for operating hours,http://hcidirectory.sg/hcidirectory/clinic.do?...
2,3,SHANG DENTAL (KALLANG),64433443.0,69541012,71 KALLANG BAHRU ...,Please call the clinic for operating hours,http://hcidirectory.sg/hcidirectory/clinic.do?...
3,4,SINGHEALTH POLYCLINICS - TAMPINES,66436969.0,69541012,1 TAMPINES STREET 41 ...,"Monday to Friday : 08:00 am to 01:00 pm, 02:00...",http://hcidirectory.sg/hcidirectory/clinic.do?...
4,5,1 MEDICAL TECK GHEE,62517030.0,69541012,BLK 410 ANG MO KIO AVENUE 10 ...,Eve of Chinese New Year (24/01/2020) : 07:00 a...,http://hcidirectory.sg/hcidirectory/clinic.do?...


In [92]:
with open('clinic_details.csv', 'w', newline='',encoding="utf-8") as file: #create csv file 
        writer = csv.writer(file,delimiter=',')
        writer.writerow(["Sno", "Name","Contact Number", "Fax","Address", 
                         "Opening Hours", "Link","Service","Programmes","Fees","Check"]) #create header
        
        for index, row in clinic_list.iterrows():
            service,program,fees = copy_details(row['Link']) #extract details
            check = t.read('//*[@id="results"]//h1')
            d_temp = []
            d_temp = [row['Sno'],row['Name'],row['Contact Number'],row['Fax'],row['Address'],
                      row['Opening Hours'],row['Link'],service,program,fees,check]      
            writer.writerow(d_temp) #write to csv file
            print (d_temp)

file.close()
global flag
flag = 0
t.close()

[1, 'DA CLINIC @ BISHAN', 69541012.0, 69541012, 'BLK 501 BISHAN STREET 11                              #01-374 Singapore 570501', 'Eve of Chinese New Year (24/01/2020) : 09:00 am to 12:00 pmEve of Chinese New Year (24/01/2020) : 08:00 am to 01:00 pm, 02:00 pm to 05:00 pmEve of Chinese New Year (24/01/2020) : 08:30 am to 12:00 pmEve of Chinese New Year (24/01/2020) : 09:00 am to 12:30 pmChinese New Year (25/01/2020) : 09:00 am to 12:00 pmChinese New Year (26/01/2020) : 09:00 am to 12:00 pm, 06:00 pm to 09:00 pmChinese New Year (26/01/2020) : 08:00 am to 01:00 pm, 02:00 pm to 05:00 pmChinese New Year (26/01/2020) : 09:00 am to 12:00 pm, 06:00 pm to 09:00 pmChinese New Year (27/01/2020) : 09:00 am to 12:00 pm, 06:00 pm to 09:00 pmChinese New Year (27/01/2020) : 09:00 am to 12:00 pm, 06:00 pm to 09:00 pmChinese New Year (27/01/2020) : 08:00 am to 01:00 pm, 02:00 pm to 05:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=mLKQpQHYV6jSJQGg0hWGQBtjPSJtLBtCb3p7vs12XMGx7dNdn0TL!-

[13, '338 FAMILY CLINIC', 64549408.0, 96707265, 'BLK 338 ANG MO KIO AVE 1                              #01-1615 Singapore 560338', 'Monday to Thursday : 08:30 am to 12:30 pm, 02:00 pm to 04:30 pm, 07:00 pm to 09:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=vFnkpT9Kth5GPPnDvMWKpX2gwTGT1STHDbrSSWx6vMKyLgRHyzJT!-774379965!284555834&task=view&toBack=search&pkId=1790', 'General Medical', 'Chronic Disease Management Programme (CDMP)\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tCommunity Health Assistance Scheme (CHAS)', 'NA', '338 FAMILY CLINIC']
[14, '8  MEDICAL AESTHETIC CLINIC', nan, 96707265, '2 HANDY ROAD                                                                    THE CATHAY                                                                                          #04-18 Singapore 229233', 'Please call the clinic for operating hours'

[25, 'A DENTAL CLINIC @ CLEMENTI CENTRAL PTE LTD', 6774168.0, 67344578, 'BLK 450 CLEMENTI AVENUE 3                              #01-285 Singapore 120450', 'Monday to Friday : 09:00 am to 01:00 pm, 02:00 pm to 05:00 pm, 06:00 pm to 09:00 pmSaturday : 09:00 am to 01:00 pm, 02:00 pm to 05:00 pmSunday and Public Holiday : 09:00 am to 01:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=mLKQpQHYV6jSJQGg0hWGQBtjPSJtLBtCb3p7vs12XMGx7dNdn0TL!-774379965!284555834&task=view&toBack=search&pkId=39600', 'General Dental\n\t\t\t\t\t\t\t  \t\n\t\t\t\t\t\t\t  \t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t  \t\n\t\t\t\t\t\t\t\t  \t\t\n\t\t\t\t\t\t\t\t\t  \t\t\n\t\t\t\t\t\t\t\t\t  \t\t\t\n\t\t\t\t\t\t\t\t\t  \t\t\t\tSpecialist Dental', 'NA', 'NA', 'A DENTAL CLINIC @ CLEMENTI CENTRAL PTE LTD']
[26, 'A DENTAL CLINIC @ JURONG EAST CENTRAL', 65690280.0, 67344578, 'BLK 135 Jurong Gateway Road                              #01-333 Singapore 600135', 'Monday to Thursday : 09:00 am to 01:00 pm, 02:00 pm t

[36, 'A+FFORDABLE CLINIC (AMK AVE 10)', 63437731.0, 62199496, 'BLK 449 ANG MO KIO AVENUE 10                              #01-1715 Singapore 560449', 'Please call the clinic for operating hours', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=mLKQpQHYV6jSJQGg0hWGQBtjPSJtLBtCb3p7vs12XMGx7dNdn0TL!-774379965!284555834&task=view&toBack=search&pkId=46881', 'General Medical', 'Community Health Assistance Scheme (CHAS)', 'NA', 'A+FFORDABLE CLINIC (AMK AVE 10)']
[37, 'A+FFORDABLE CLINIC (HOUGANG)', 65849388.0, 62199496, 'BLK 681 HOUGANG AVENUE 8                              #01-829 Singapore 530681', '24 Hour', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=mLKQpQHYV6jSJQGg0hWGQBtjPSJtLBtCb3p7vs12XMGx7dNdn0TL!-774379965!284555834&task=view&toBack=search&pkId=44918', 'General Medical', 'Chronic Disease Management Programme (CDMP)\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\

[49, 'ABSOLUTE SMILE DENTAL', nan, 63112905, '2 TAI THONG CRESCENT                                                                    THE VENUE SHOPPES                                                                                          #01-04 Singapore 347836', 'Please call the clinic for operating hours', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=mLKQpQHYV6jSJQGg0hWGQBtjPSJtLBtCb3p7vs12XMGx7dNdn0TL!-774379965!284555834&task=view&toBack=search&pkId=46686', 'General Dental', 'NA', 'NA', 'ABSOLUTE SMILE DENTAL']
[50, 'ACACIA MEDICAL CLINIC', 98164823.0, 63112905, '180 DEPOT ROAD                                                                    INTERLACE, THE                                                                                          #01-07 Singapore 109684', 'Eve of Lunar New Year (04/02/2019) : 09:00 am to 12:00 pmLunar New Year (05/02/2019) : 06:00 pm to 09:00 pmLunar New Year (06/02/2019) : 06:00 pm to 09:00 pm', 'http://hcidirectory.sg/hcidirectory/

...

[4364, 'YAP FAMILY CLINIC AND SURGERY PTE. LTD.', 64577618.0, 67830232, 'BLK 232 ANG MO KIO AVE 3                              #01-1208 Singapore 560232', 'Eve of Chinese New Year (24/01/2020) : 08:30 am to 12:00 pmMonday to Friday : 08:30 am to 12:30 pm, 02:00 pm to 04:30 pm, 07:00 pm to 09:00 pmSaturday : 08:30 am to 12:30 pm, 07:00 pm to 09:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!NONE&task=view&toBack=search&pkId=30530', 'General Medical', 'Chronic Disease Management Programme (CDMP)', 'NA', 'YAP FAMILY CLINIC AND SURGERY PTE. LTD.']
[4365, 'YAP MEDICARE CLINIC & SURGERY', 64712338.0, 67830232, 'BLK 8 EMPRESS RD                              #01-03 Singapore 260008', 'Eve of Chinese New Year (24/01/2020) : 08:30 am to 12:00 pmMonday to Friday : 08:30 am to 12:00 pm, 02:00 pm to 04:00 pm, 07:00 pm to 08:45 pmSaturday and Sunday : 08:30 am to 12:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.

[4374, 'YEO ORTHOPAEDIC CENTRE', 67349394.0, 62868633, '3 MT ELIZABETH                                                                    MOUNT ELIZABETH MEDICAL CENTRE                                                                                          #17-09 Singapore 228510', 'Please call the clinic for operating hours', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!NONE&task=view&toBack=search&pkId=30640', 'Specialist Medical', 'NA', 'NA', 'YEO ORTHOPAEDIC CENTRE']
[4375, 'YEO SEEM HUAT PSYCHIATRIC CLINIC PTE LTD', 67345537.0, 62868633, '290 ORCHARD ROAD                                                                    PARAGON                                                                                          #06-10 Singapore 238859', 'Monday to Friday : 08:30 am to 04:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!

[4385, 'YIO CHU KANG NAM SENG DENTIST', 64595418.0, 62732316, 'BLK 603 ANG MO KIO AVE 5                              #01-2669 Singapore 560603', 'Public Holiday :  ClosedMonday to Friday : 09:00 am to 12:00 pm, 02:00 pm to 05:00 pmSaturday : 09:00 am to 12:00 pmSunday :  Closed', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!NONE&task=view&toBack=search&pkId=30770', 'General Dental', 'NA', 'NA', 'YIO CHU KANG NAM SENG DENTIST']
[4386, 'YIP DENTAL SURGERY PTE LTD', 62354477.0, 62732316, '3 MT ELIZABETH                                                                    MOUNT ELIZABETH MEDICAL CENTRE                                                                                          #04-01 Singapore 228510', 'Monday to Saturday : 08:30 am to 06:00 pm', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!NONE&task=view&toBack=search&pkId=307

[4396, 'YSL ALJUNIED CLINIC & SURGERY', 63430626.0, 62577202, 'BLK 5 UPPER ALJUNIED LANE                              #01-42 Singapore 360005', 'Eve of Chinese New Year (24/01/2020) : 08:30 am to 12:30 am', 'http://hcidirectory.sg/hcidirectory/clinic.do?JSESSIONID=7V9npQQDFpyh0myQqZR9hNw9WKTBZhXfYSJ72PTKXjjknvr2t7zk!-774379965!NONE&task=view&toBack=search&pkId=45713', 'General Medical', 'Chronic Disease Management Programme (CDMP)\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\tCommunity Health Assistance Scheme (CHAS)', 'NA', 'YSL ALJUNIED CLINIC & SURGERY']
[4397, 'YSL BEDOK CLINIC & SURGERY', 62452684.0, 62577202, 'BLK 539 BEDOK NORTH STREET 3                              #01-631 Singapore 460539', 'Eve of Chinese New Year (24/01/2020) : 08:00 am to 01:00 pmMonday to Friday : 08:00 am to 12:30 am, 02:00 pm to 05:00 pm, 07:00 pm to 10:00 pmSaturday and Sunday

True